In [2]:
!pip install python-dotenv
!pip install huggingface_hub

In [3]:
from huggingface_hub import login

login()

## Load the Model and Tokenizer from Local

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "11andrea2233"
model_id = f"{username}/gemma-2b-instruct-ft-ai-medical"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Test the finetuned model

In [5]:
# Helper functions to get the responses in proper format

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
   What does abutment of the nerve root mean? 
  
model


  Hi, I have gone through your query, based on the given scenario your MRI report is suggestive of root impingement at the L5-S1 level with neurogenic claudication. It is an injury to the nerve root that causes compression of its root and can induce pain. You will benefit by managing this with conservative management like pain killers and physiotherapy and in some cases with  the use of a small corticosteroid like injection at the spinal cord by a neurologist. Hope this information will help you. For more clarification consult a neurologist over the phone or online. Good Luck Dr Nikhil Shah neurologist. 


In [6]:
query = """ What does abutment of the nerve root mean? """

result = get_completion(query = query, model = model, tokenizer = tokenizer)
print(result)


  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
   What does abutment of the nerve root mean? 
  
model


  abtment of the nerve root usually means injury/problem in the nerve root.  its cause is trauma, infection, spinal stenosis etc. In this case it will affect the nerve root and cause problem such as pain, numbness etc on the affected nerve root. 


In [14]:
print(f"{result.split('Analysis:')[-1]}")


  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
   What does abutment of the nerve root mean? 
  
model


  abtment of the nerve root usually means injury/problem in the nerve root.  its cause is trauma, infection, spinal stenosis etc. In this case it will affect the nerve root and cause problem such as pain, numbness etc on the affected nerve root. 
